# Sentiment Analysis

This notebook generate the sentiment analysis results and save it in UC for downstream use. 

# Load Data

In [0]:
# Standard library imports
import textwrap

# Third-party imports
import pandas as pd
import mlflow
from transformers import pipeline

# Local application imports
import de_utils as dut

In [0]:
df_spark = spark.sql("SELECT * FROM bupa_call_synthetic_dataset")
df = df_spark.toPandas()

In [0]:
df.head().display()

CallID,MembershipNumber,AgentID,DatetimeStamp,AppTag,FirstContactResolution,NPS,NCR,Summary
20250505000001,68682677,christopher.brown@bupa.com.au,2025-05-05T08:00:03.000Z,Claim Status,1,9,10,"The member called to inquire about claim status. The member reported that their recent hospital claim had not appeared in their online portal and they were unsure about the payment status. The agent checked the claim records, confirmed that the claim was processed yesterday, and guided the member to refresh their portal cache to view the updated status. Information sent via email. Detailed explanations helped build the member�s confidence. They provided clear guidance on the next steps. The agent ensured all questions were answered thoroughly. They provided clear guidance on the next steps. Detailed explanations helped build the member�s confidence. Detailed explanations helped build the member�s confidence. The agent actively listened and showed empathy throughout the call. The agent ensured all questions were answered thoroughly. The agent ensured all questions were answered thoroughly. They provided clear guidance on the next steps. They provided clear guidance on the next steps. The member appreciated the agent�s patience and professionalism. The agent actively listened and showed empathy throughout the call. The agent ensured all questions were answered thoroughly. They provided clear guidance on the next steps. They provided clear guidance on the next steps. Detailed explanations helped build the member�s confidence. They provided clear guidance on the next steps. They provided clear guidance on the next steps. The agent ensured all questions were answered thoroughly. The agent actively listened and showed empathy throughout the call. The agent ensured all questions were answered thoroughly. The member appreciated the agent�s patience and professionalism. Detailed explanations helped build the member�s confidence."
20250505000002,17327472,christopher.brown@bupa.com.au,2025-05-05T08:00:04.000Z,Policy Change,1,2,3,"The member called to inquire about policy change. The member wanted to add a newborn to their family policy but was unclear about the effective date and premium difference. The agent updated the policy to include the newborn from the date of birth, recalculated the premium, and emailed the updated policy documents to the member. Resolved on call. They provided clear guidance on the next steps. The agent actively listened and showed empathy throughout the call. Detailed explanations helped build the member�s confidence. The agent ensured all questions were answered thoroughly. The agent actively listened and showed empathy throughout the call. The agent actively listened and showed empathy throughout the call. The agent ensured all questions were answered thoroughly. The agent actively listened and showed empathy throughout the call. The agent actively listened and showed empathy throughout the call. The agent ensured all questions were answered thoroughly. They provided clear guidance on the next steps. The member appreciated the agent�s patience and professionalism. The member appreciated the agent�s patience and professionalism. Detailed explanations helped build the member�s confidence. The agent actively listened and showed empathy throughout the call. The member appreciated the agent�s patience and professionalism. The agent actively listened and showed empathy throughout the call. The agent ensured all questions were answered thoroughly. The agent actively listened and showed empathy throughout the call. The agent ensured all questions were answered thoroughly. Detailed explanations helped build the member�s confidence. Detailed explanations helped build the member�s confidence. The member appreciated the agent�s patience and professionalism."
20250505000003,41303110,sarah.brown@bupa.com.au,2025-05-05T08:00:05.000Z,Claim Status,0,6,9,"The member called to inquire about claim status. The member reported that their rece

# Sentiment Analysis

In [0]:
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    framework="pt"
)

def summarize_sentiment(text, chunk_size=500):
    # Break the text into chunks
    chunks = textwrap.wrap(text, chunk_size)
    
    # Initialize variables to store scores
    scores = []
    first_chunk_score = None
    last_chunk_score = None
    
    # Analyze each chunk
    for i, chunk in enumerate(chunks):
        result = sentiment_analyzer(chunk)
        score = result[0]['score']
        scores.append(score)
        
        # Capture scores for the first and last chunks
        if i == 0:
            first_chunk_score = score
        if i == len(chunks) - 1:
            last_chunk_score = score
    
    # Calculate the average score
    average_score = sum(scores) / len(scores) if scores else 0.0
    
    return {
        'average_score': average_score,
        'first_chunk_score': first_chunk_score,
        'last_chunk_score': last_chunk_score
    }

In [0]:
# Implement the analysis 
sentiment_scores = df['Summary'].apply(summarize_sentiment)
df[['average_score', 'first_chunk_score', 'last_chunk_score']] = pd.DataFrame(sentiment_scores.tolist(), index=df.index)

In [0]:
# Write the data back to UC
table_name = "bupa_call_synthetic_dataset_with_sentiment_score"
dut.write_data_to_bricks_catalog(df, table_name)